## 1. 初始準備

In [1]:
%env KERAS_BACKEND=tensorflow
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

# Keras functions
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

# Keras dataset
from keras.datasets import mnist

# Keras utils function
from keras.utils import np_utils

env: KERAS_BACKEND=tensorflow


讀取 MNIST 手寫辨識資料

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

In [4]:
print(x_train.shape, x_test.shape)

(60000, 784) (10000, 784)


In [5]:
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

## 2. Model Function API
在此之前使用 Sequential 便足以建構大多數的神經網路，那是因為接觸的神經網路多為線性堆疊 (linear stack)
除了輸入曾需指定 `input_dim` 外，其餘隱藏曾只需宣告，那是因為 Sequential 會認定上一層的輸出這一層的輸入
因此在建構線性堆疊的神經網路時，Sequential 便足以處理

### 2.1 Functional API 的使用時機
當神經網路模型為非線性的複雜網路結構，如：
* 多重輸出-多重輸入模型 (Multi-input and multi-output models)
    * 分歧 (branch)
    * 合併 (merge)
* 具重複/循環結構的模型，如：CycleGAN

Sequential 便不足以建構這類複雜結構的神經網路，以下介紹 `Model` Functional API 的使用

首先將 `Model` 引入

In [6]:
from keras.models import Model

在 `Model` 的世界中，所有的神經網路層 (fully-connected, convolution, MaxPooling, LSTM, etc) 都被視作函數來操作，因此只需關心函數的輸入和輸出即可

此外，為了讓神經網路的第一層從不需要輸入 `input_dim`，我們還需進下面這個函數來代替 `input_dim` (此寫法亦可用在 `Sequential`)

In [7]:
# 把輸入層讀進來
from keras.layers import Input

### 2.2 Functional API 的函數概念
$R^{784} -> R^{10}$

* 注意：為了方便，將 `Dense(500)`，`Activation('sigmoid')` 兩個合併用 `Dense(500, activation = 'sigmoid')` 表示

### 2.3 Function API 的操作方式

In [8]:
f_1 = Dense(500, activation='sigmoid')
f_2 = Dense(500, activation='sigmoid')
f_3 = Dense(10, activation='softmax')

In [9]:
print(f_1)

接著定義層前後變數之間的關係，首先第一個變數必定以 `Input` 函數來定義

In [10]:
x = Input(shape=(784,))

In [11]:
print(x)

Tensor("input_1:0", shape=(?, 784), dtype=float32)


剩下的部分可以照著數學式輸入

$h_1 = f_1(x),h_2 = f_2(h_1),y=f_3(h_2)$

In [12]:
h_1 = f_1(x)
h_2 = f_2(h_1)
y = f_3(h_2)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


在這裡，變數 $h_1$, $h_2$, $y$ 是以張量 (tensor) 類別來表示，可以嘗試 `print` 看看

In [13]:
print(h_1)
print(h_2)
print(y)

Tensor("dense_1/Sigmoid:0", shape=(?, 500), dtype=float32)
Tensor("dense_2/Sigmoid:0", shape=(?, 500), dtype=float32)
Tensor("dense_3/Softmax:0", shape=(?, 10), dtype=float32)


接著，透過 `Model` 將一個模型的輸入 / 輸出包裝起來，建立模型的過程就完成了

In [14]:
model = Model(x, y)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               392500    
_________________________________________________________________
dense_2 (Dense)              (None, 500)               250500    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5010      
Total params: 648,010
Trainable params: 648,010
Non-trainable params: 0
_________________________________________________________________


一樣的，當模型 compile 後，便可以進行資料的訓練、預測...等，讀入 MNIST 手寫辨識資料後完成模型的訓練

In [15]:
model.compile(loss='mse', optimizer=SGD(lr=0.1), metrics=['accuracy'])

In [18]:
model.fit(x_train, y_train, batch_size=100, epochs=5)


Epoch 1/5
60000/60000 [==============================] - 11s 177us/step - loss: 0.0830 - accuracy: 0.3665
Epoch 2/5
60000/60000 [==============================] - 21s 350us/step - loss: 0.0628 - accuracy: 0.6474
Epoch 3/5
60000/60000 [==============================] - 15s 253us/step - loss: 0.0449 - accuracy: 0.7643
Epoch 4/5
60000/60000 [==============================] - 17s 276us/step - loss: 0.0332 - accuracy: 0.8401
Epoch 5/5
60000/60000 [==============================] - 15s 258us/step - loss: 0.0259 - accuracy: 0.8726


In [19]:
model.load_weights('handwriteing_model_weights.h5')

In [20]:
model.evaluate(x_test, y_test)

10000/10000 [==============================] - 1s 147us/step


[0.010523480594716967, 0.9343000054359436]

### 2.4 結論
Functional API 的操作流程：
1. 將層定義成明確的函數
2. 透過層函數將變數連接
3. 定義神經網路的輸入與輸出

## 3. 非線性堆疊模型
### 3.1 如果建立具分歧及合併結構的神經網路模型？

In [ ]:
from keras.layers import concatenate, add